# Milestone 2 - Data gathering and preprocessing

## 1. Narrowing down the research question

### Research Questions

In this research project we will try to analyse a corpus of popular songs to try to identify  chords differences between verses and choruses. We will try to  answer the following research questions :
<ol>
<li>Does the chord distribution of the choruses differ from the one in the verses?</li>
<li>Is there a different chord sequence distribution in the choruses compared to the verses?</li>
<li>How are these distributions evolving over time? </li>
</ol>

In the first question we want to know if a difference exists in terms of chords statistic. Are we more likely to find a specific chord in the choruses? Are fewer different chords used in the chorus compared to the other parts of a song? <br> 
For the second one we want to have a more melodic insight. Can we find specific patterns? Is the Markov model derived from the chorus different from its counterparts? <br>
In the third one the focus is on the time dimension. We want to know if the results of the two previous answers change over time. Were chorus closer to verses in 1968 than in 1985? How does each chord evolve in relation to the others? Do we find a sequence that appeared while other disappeared? 

These are all the underlying questions we want to answer under our main research questions.  

These questions relate to our original idea to fully characterize a chorus over the years, especially in comparison with verses. We now want to focus specifically on the chords to differentiate these different parts, while keeping the temporal dimension as a potential factor to observe changes.

### Dataset presentation

To give us the means to answer our questions, we have selected a dataset containing approximately 900 Pop-Rock songs in the top Billboard charts from the 60s to the 90s. They are simple text files with the following informations : 
* Release date
* Song title 
* Artist name
* Labels for the different parts of the song (such as chorus or verse)
* Timestamp of each musical phrase beginning
* Chords

### Procedure

We will use the chorus/verse annotations to classify the chords in each group. This will allow us to divide the chords in two groups and compute statistics and distributions for each of them. As discussed in the research questions, we will start with a basic characterization, simply comparing which chords appear in which section. We will then move deeper and compare the distributions of chords as well as the Markov models. These will be computed at least with bigrams, perhaps with higher-n n-grams depending on the number of chords in each section (as it makes little sense to use n-grams with n close to the number of chords in a given section).

The metadata, especially the release date, will be used to study the evolution over time of the previously discussed statistics. Depending on the distribution of songs over the years, time analysis will be discussed either over years or over decades.

### Possible outcomes and confidence measures

The different outcomes we can reasonably expect are: 
<ul>
    <li> <strong>Null results:</strong> There are no significant differences between choruses and verses and no evolution over time. This could be explained by a bias in the corpus toward a specific Pop-Rock genre using the same chords all the time or maybe there is indeed no difference in the chords used in a chorus and the ones used in verses, which would constitute an answer for our questions. </li>
<li> <strong>Narrow chorus chord distribution:</strong> Since the chorus has to be immediately recognized as one, maybe the composers make more extensive use of a sub group of the chords to ensure it. The same reasoning could be applied to the chord sequences: perhaps some specific ones will be more dominant in the chorus.</li>
<li> <strong>Temporal evolution:</strong> It will be interesting to see if the differences between verse and chorus change over the years. This evolution, if present, could be linear or oscillating. A linear narrowing would imply that choruses are becoming more and more similar or verse more and more diverse. An oscillating pattern would be interesting as musical phenomenon could appear and disappear.</li>
</ul>

Statistical tests will be used throughtout our analysis to check if our findings are statistically relevant. Error bars will also be included in all our graphics to avoid wrong conclusions. Of course the relative small size of our corpus will influence our results but only further analysis can reveal if significant results can still be found.

## 2. Gathering the data
The dataset has been created by [1] and corresponds to a random sample of 890 Billboard chart slots presented at ISMIR 2011 and MIREX 2012. Due to the nature of the sampling algorithm, there are some duplicates and this results in only 740 distinct songs. According to the authors, training algorithms that assume independent, identically distributed data should retain the duplicates.<br> This dataset is publicly available at https://ddmal.music.mcgill.ca/research/The_McGill_Billboard_Project_(Chord_Analysis_Dataset)/ and can be downloaded in various formats. Different features are given by the authors. In this project we will use metadata and chords annotations. 
The first dataset used is the index to the dataset (csv format), containing the following fields:
<ul>
<li><b>id</b>, the index for the sample entry.</li>
<li><b>chart_date</b>, the date of the chart for the entry.</li>
<li><b>target_rank</b>, the desired rank on that chart.</li>
<li><b>actual_rank</b>, the rank of the song actually annotated, which may be up to 2 ranks higher or lower than the target rank [1, 2].</li>
<li><b>title</b>, the title of the song annotated.</li>
<li><b>artist</b>, the name of the artist performing the song annotated.</li>
<li><b>peak_rank</b>, the highest rank the song annotated ever achieved on the Billboard Hot 100.</li>
<li><b>weeks_on_chart</b>, the number of weeks the song annotated spent on the Billboard Hot 100 chart in total.</li>
</ul>

The main dataset comprehends chords, structure, instrumentation, and timing, given in a txt format. The annotation for each song begins with a header containing the title of the song, the name of the artist, the metre and the tonic pitch class of the opening key. In the main body, each line consists of a single phrase and begins with its timestamp, followed by the chords. This requires us to design a specific parser, as will be discussed in the next section.<br>
We downloaded the two datasets which constitutes the whole of their database so we have the maximum from this source. It is not excluded that we find some additional ones to perform further analysis. As for now we will try to get as much stastitically relevant information from this source.

[1]: John Ashley Burgoyne, Jonathan Wild, and Ichiro Fujinaga, ‘An Expert Ground Truth Set for Audio Chord Recognition and Music Analysis’, in Proceedings of the 12th International Society for Music Information Retrieval Conference, ed. Anssi Klapuri and Colby Leider (Miami, FL, 2011), pp. 633–38

## 3. Data format

The goal of this question is to load the data and have a look at it. A specific parser is designed to do this automatically, in order to extract and store in a Pandas dataframe all the relevant informations and musical features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='white')
import plotly.express as px
import seaborn as sns

### Metadata

In [ ]:
metadata_df = pd.read_csv("data/billboard-2.0-index.csv")
metadata_df.head(10)

In [ ]:
print('There are %d entries in the index table.' %len(metadata_df))

In [ ]:
print('There are %d entries with a given title.' %metadata_df.title.isna().value_counts()[0])

In [ ]:
print('There are %d entries with a given artist.' %metadata_df.artist.isna().value_counts()[0])

In [ ]:
print('There are %d entries with a given chart date.' %metadata_df.chart_date.isna().value_counts()[0])

In [ ]:
months = {'01':'January', 
          '02':'February',
          '03':'March',
          '04':'April',
          '05':'May',
          '06':'June',
          '07':'July',
          '08':'August',
          '09':'September',
          '10':'October',
          '11':'November',
          '12':'December'}
def format_date(date):
    year = date[:4]
    month = date[5:7]
    day = date[-2:]
    if day == '01':
        suffix = 'st'
    elif day == '02':
        suffix = 'nd'
    elif day == '03':
        suffix = 'rd'
    else:
        suffix = 'th'
        
    if day[0] == '0':
        day = day[1]
    
    date_string = months[month] + ' ' + day + suffix + ', ' + year
    return(date_string)

#Test
format_date('1958-08-04')

In [ ]:
print('The songs range from %s to %s.' %(format_date(metadata_df.chart_date.min()), format_date(metadata_df.chart_date.max())))

### Parser 

In [ ]:
SONG_ID, LINE_NUMBER, MEASURE_NUMBER, CHORD_NUMBER, SEQUENCE_NUMBER, \
CHORD, INSTRUMENT, TYPE, TIME, STRUCTURE, DURATION, REPETITION, ELID = \
"song_id","line_id", "measure_id", "chord_id", "sequence_id",\
"chord", "instrument", "section_type", "time", "section_structure", "duration", "repetition", "elided"

#This is dependant of "metre" in the txt files.
METRE = "metre"

#Create a new dictionary from two other
def immutable_merge(dic1, dic2):
    result = dic1.copy()
    result.update(dic2)
    return result

#Create a row of the futur df as a dictionary
def create_row(persistent_attributes, line_attributes,
               measure_number = None, chord_number = None, chord = None, duration = None):
    result = immutable_merge(persistent_attributes, line_attributes)
    
    if not (measure_number is None and measure_number is None and chord_number is None and duration is None):
        result[MEASURE_NUMBER] = measure_number
        result[CHORD_NUMBER] = chord_number
        result[CHORD] = chord
        result[DURATION] = duration
    
    return result

#Generate the attributes of a given line and update the sequence counter
def process_line_metadata(header, line_counter, old_line_attributes, sequence_counter, suffix = ""):
    
    result = {}
    
    #Suffix (main instrument, elid, repetition)
    old_instrument = str(old_line_attributes.get(INSTRUMENT))
    
    for suffix in suffix.split(", "):
        
        suffix = suffix.strip()
        
        #Repetition
        if re.match("^x\d+$",suffix):
            result[REPETITION] = int(suffix[1])
        
        #Elid
        elif suffix == "->":
            result[ELID] = True

        #Instrument
        else:
            ##New instrument
            if len(suffix) > 0 and suffix != "\n":
                result[INSTRUMENT] = suffix.strip("\n").strip(",").strip()

            ##Main instrument continued (experimental)
            elif not old_instrument.endswith(")") and old_instrument.lower() not in ["nan","none"] \
            and len(old_instrument)>0:
                result[INSTRUMENT] = old_instrument.strip("(")

        
    #Line number
    result[LINE_NUMBER] = line_counter

    
    #Header    
    header_items = header.split()
        
    result[TIME] = header_items[0]
    
    #Case where a section is continued
    if len(header_items) == 1:
        result[TYPE] = old_line_attributes.get(TYPE)
        result[STRUCTURE] = old_line_attributes.get(STRUCTURE)
        result[SEQUENCE_NUMBER] = old_line_attributes.get(SEQUENCE_NUMBER)
    
    #Case where a section has no structure (silence, end, fadeout)
    elif len(header_items) == 2:
        
        #Z is a structure, not a type.
        if header_items[1].strip().strip(",") == "Z":
            result[STRUCTURE] = header_items[1].strip().strip(",")
        else:
            result[TYPE] = header_items[1].strip().strip(",")
            
        result[SEQUENCE_NUMBER] = sequence_counter
        sequence_counter += 1
    
    #Case where a section begins.
    elif len(header_items) == 3:
        result[STRUCTURE] = header_items[1].strip().strip(",")
        result[TYPE] = header_items[2].strip().strip(",")
        result[SEQUENCE_NUMBER] = sequence_counter
        sequence_counter += 1
    
    return sequence_counter, result

In [ ]:
def parse_song_to_dict(song_id, path):
    
    rows = []
    persistent_attributes = {}
    
    persistent_attributes[SONG_ID] = song_id
    
    with open(path,"r") as file:
        line = file.readline()
        
        line_counter = 0
        measure_counter = 0
        chord_counter = 0
        sequence_counter = 0
        line_attributes = {}
        old_chord = None
 

        while line:
        
            if line != "\n":

                #Attribute lines
                if line.startswith("#"):
                    attribute, value = line.strip("#").split(":",1)
                    persistent_attributes[attribute.strip(" ")] = value.strip(" ").strip("\n")

                else:
                    line_items = line.split("|")

                    #Special lines
                    if len(line_items) <= 1:
                        sequence_counter, line_attributes = \
                        process_line_metadata(line, line_counter, line_attributes, sequence_counter)
                        row = create_row(persistent_attributes, line_attributes)
                        rows.append(row)

                    #Standard lines    
                    else:                    
                        header = line_items[0]
                        suffix = line_items[-1]
                        measures = line_items[1:-1]

                        sequence_counter, line_attributes = \
                        process_line_metadata(header, line_counter, line_attributes, sequence_counter, suffix)  

                        for measure in measures:
                            
                            chords = measure.split()
                            
                            #Special metric (experimental)
                            old_metre = persistent_attributes.get(METRE)
                            if re.match("^\(\d/\d\)$", chords[0]):
                                persistent_attributes[METRE] = str(chords[0][1]) + "/" + str(chords[0][3])
                                chords = chords[1:]
                            
                            if len(chords) == 1:
                                duration = "measure"
                            elif len(chords) == 2 and persistent_attributes[METRE] in ["4/4","12/8"]:
                                duration = "half-measure"
                            else:
                                duration = "beat"
                            
                            for chord in chords:
                                
                                if chord == ".":
                                    chord = old_chord
                                
                                row = create_row(persistent_attributes, line_attributes,
                                                 measure_counter, chord_counter, chord, duration)
                                rows.append(row)
                                old_chord = chord
                                chord_counter += 1

                            measure_counter += 1
                            persistent_attributes[METRE] = old_metre
            
            #Finally
            line_counter += 1
            line = file.readline()
    
    
    return rows

In [ ]:
test = pd.DataFrame(parse_song_to_dict(0,"data/McGill-Billboard/0004/salami_chords.txt"))

In [ ]:
def create_whole_collection_df():
    
    path = "data/McGill-Billboard/"
    file_name = "/salami_chords.txt"
    UPPER_BOUND = 1300
    
    whole_collection = []
    
    i = 0
    while i <= UPPER_BOUND:
        full_path = path + "0"*(4-len(str(i)))+ str(i) + file_name
        
        if os.path.exists(full_path):
            whole_collection += parse_song_to_dict(i, full_path)
        
        i += 1
        
    whole_collection_df = pd.DataFrame(whole_collection)
    
    return whole_collection_df.astype({SEQUENCE_NUMBER: 'Int64', MEASURE_NUMBER: 'Int64', CHORD_NUMBER: 'Int64', \
                                      REPETITION: 'Int64'})

In [ ]:
collection_df = create_whole_collection_df()

In [ ]:
collection_df.sample(10)

### Exploratory statistical analysis

If we look at the number of sample per year we have, we see it's pretty well distributed. 

In [ ]:
years=metadata_df.chart_date.map(lambda y:pd.to_datetime(y).year).value_counts(sort=False)
years.iplot(kind='bar', title="Number of sample song per year", xTitle="Year", yTitle="Samples")




In [ ]:
collection_df.chord.value_counts().head(24).iplot(kind='bar',title="Overall chord occurence ditribution",xTitle="Chord",yTitle="Occurence")

In [ ]:
collection_df[collection_df["section_type"]=="chorus"].chord.value_counts().head(20).iplot(kind='bar',title="Chorus chord occurence ditribution",xTitle="Chord",yTitle="Occurence")

In [ ]:
print("Number of songs that contain a chorus : %s" %str(np.sum(collection_df[[SONG_ID,TYPE]].drop_duplicates()[TYPE] == "chorus")))

print("Number of songs that contain a verse : %s" %str(np.sum(collection_df[[SONG_ID,TYPE]].drop_duplicates()[TYPE] == "verse")))

unique_section_songs = collection_df[[SONG_ID,TYPE]].drop_duplicates().groupby(SONG_ID)[TYPE].apply(list)
print("Number of songs that contain both : %s" %str(np.sum(unique_section_songs.apply(lambda l : "chorus" in l and "verse" in l))))

In [ ]:
#Basic statistics: Number of unique chords per songs
unique_chord_songs = collection_df[[SONG_ID,CHORD]].drop_duplicates().groupby(SONG_ID).count()
n_bins = int(unique_chord_songs.max())+1
unique_chord_songs.plot.hist(bins = n_bins, legend = False)
plt.title('Distribution of the number of unique chords per song', fontsize = 20)
plt.xlabel('Number of unique chord', fontsize = 18)
plt.ylabel('Frequency', fontsize = 18)
plt.show()

# Milestone 3

<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Final processing</strong></div>
    <div>
        The first step for this milestone is to finish the processing of our dataset in order to obtain a dataframe containing all the chords of all songs that have both a verse and a chorus. We also add a parser for the chord notation used to better understand it. Thus we will:
            <ul>
                <li>parse the notation of the chords used in the dataset</li>
                <li>implement the relative-to-tonic pitch class</li>
                <li>discard all songs without both a verse and a chorus</li>
                <li>split the dataframe to have a row per beat</li>
                <li>create a function to squeeze all chords of a song in a list</li>
            </ul>
    </div>
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Parser for the chords</strong>
</div> 

We also define a parser to better handle the chords. This is based on the notation defined in http://ismir2005.ismir.net/proceedings/1080.pdf. Some notations are also added by the authors of the dataset and therefore included to handle all the 989 unique chords in the dataset. 

In [ ]:
print('There are %d unique chords in the dataset.' %len(collection_df.chord.unique()))

In [ ]:
#Split a string and changes the target field depending on which side of the split to take
#side : 0 for left and 1 for right
def split_add(c,dic,split_char,target,side):
    temp=str.split(str(c),split_char)

    dic[target]=temp[side]
    return dic, temp[1-side]


def chord_to_tab(c):
    c=str(c)
    chord={"root":"", "shorthand" : "", "degree_list":[], "bass":"", "N" :False}
    rest=""
    
    if(c=="N"):
        chord["N"] = True
        return chord
    
    c=c.replace(")","")
    
    if('/' in c):
        chord, rest=split_add(c,dic=chord,split_char="/",target="bass",side=1)
    else :
        rest=c
    if(':' in rest):
        chord, rest=split_add(rest,dic=chord,split_char=":",target="root",side=0)
    if('(' in rest):
        chord, rest=split_add(rest,dic=chord,split_char="(",target="degree_list",side=1)
    if(rest != ""):
        chord["shorthand"]=rest
    
    return chord

    

In [ ]:
collection_df['chord_dic'] = collection_df.chord.map(lambda y : chord_to_tab(y))
for key in ['root', 'shorthand', 'degree_list', 'bass', 'N']:
    collection_df[key] = collection_df.chord_dic.map(lambda y: y[key])
collection_df.drop(columns = 'chord_dic', inplace = True)
#df = collection_df[['song_id', 'title', 'artist', 'tonic', 'section_type', 'sequence_id', 'chord', 'root', 'shorthand', 'degree_list', 'bass', 'N']]

In [ ]:
collection_df.head()

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Creation of new columns fo relative-to-tonic roots</strong>
</div> 

In [ ]:
TPC_DIC = {"Cb":11,"C":0,"C#":1,"Db":1,"D":2,"D#":3,"Eb":3,"E":4,"E#":5,"Fb":4,"F":5,"F#":6,
           "Gb":6,"G":7,"G#":8,"Ab":8,"A":9,"A#":10,"Bb":10,"B":11,"B#":0}

In [ ]:
collection_df["root_tpc"] = collection_df.root.apply(lambda r : TPC_DIC.get(r))
collection_df = collection_df.astype({"root_tpc":"Int64"})

In [ ]:
collection_df["relative_root_tpc"] =\
collection_df.apply(lambda row: (row["root_tpc"] - TPC_DIC.get(row["tonic"]))%12,axis = 1)

In [ ]:
collection_df = collection_df.fillna({REPETITION:1})

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Creation of the dataframe with only songs that have both a verse and a chorus</strong>
</div> 

In [ ]:
valid_songs = collection_df[[SONG_ID,TYPE]].drop_duplicates().groupby(SONG_ID)[TYPE].apply(list)\
.apply(lambda l: "chorus" in l and "verse" in l)

In [ ]:
d_collection_df = collection_df.merge(valid_songs.reset_index().rename(columns = {TYPE:"valid"}),on = SONG_ID)
d_collection_df = d_collection_df[d_collection_df.valid].drop(columns = 'valid')

In [ ]:
print('The corpus has %d songs.' %len(d_collection_df.song_id.unique()))

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Creation of a one-row-per-beat dataframe</strong>
</div> 

Duration of measure: all metre in the corpus are regular except song 700 (5/8 = 3 + 2). This was counted as two beats.

In [ ]:
def weight_row(metre, duration, repetition):
    """
    Indicate to how many beats in total a chord described on a dataframe's row correspond.
    Careful ! The chords are non-successive in case of repetition
    """
    num, denom = metre.split("/")
    
    if metre == "5/8": #Specific to this dataset
        beat_per_measure = 2
    else:
        beat_per_measure = int(num)/(1 if int(denom) == 4 else 3)
    
    if duration == "measure":
        return beat_per_measure*(repetition)
    
    elif duration == "half-measure":
        return beat_per_measure/2*(repetition)
    
    elif duration == "beat":
        return repetition
    
    else:
        return 0

def successive_repetition_row(metre,duration):
    """
    Indicate on how many successive beats a chord is present
    """
    return weight_row(metre,duration,repetition=1)

In [ ]:
# Creation of one-row-per-bit dataframe
N_SUCC_BEATS = "n_succ_beats"

d_collection_df[N_SUCC_BEATS] = d_collection_df.apply(\
    lambda row: successive_repetition_row(row[METRE], row[DURATION]),axis=1)
d_collection_df = d_collection_df.astype({N_SUCC_BEATS:"Int64"})

In [ ]:
from tqdm import tqdm

def create_beats_df(d_collection_df):
    beats_dics = []
    repetition_flag = False
    repeted_dics = []
    repetition_line = np.PINF
    repetition_song = np.PINF
    for i in tqdm(d_collection_df.reset_index().index):

        if repetition_flag == True and\
(repetition_line != d_collection_df.iloc[i][LINE_NUMBER] or repetition_song != d_collection_df.iloc[i][SONG_ID]):

            for r in range(repetition_n):
                beats_dics += repeted_dics

            repetition_flag = False
            repeted_dics = []
            repetition_line = np.PINF
            repetition_song = np.PINF
        
        
        if d_collection_df.iloc[i][REPETITION] == 1 :

            for b in range(d_collection_df.iloc[i][N_SUCC_BEATS]):
                beats_dics.append(d_collection_df.iloc[i].to_dict())

        else:

            repetition_flag = True
            repetition_line = d_collection_df.iloc[i][LINE_NUMBER]
            repetition_song = d_collection_df.iloc[i][SONG_ID]
            repetition_n = d_collection_df.iloc[i][REPETITION]

            for b in range(d_collection_df.iloc[i][N_SUCC_BEATS]):
                repeted_dics.append(d_collection_df.iloc[i].to_dict())

    beats_collection_df = pd.DataFrame(beats_dics)
    
    return beats_collection_df

In [ ]:
beats_collection_df = create_beats_df(d_collection_df)
beats_collection_df = beats_collection_df.drop(N_SUCC_BEATS,axis=1)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> the final dataframe beats_collection_df has the following properties:
    <ul>
        <li>each row corresponds to a beat and thus to a chord played</li>
        <li>only songs with both a verse and a chorus are kept</li>
        <li>all elements of a chord (root, quality, degree list and bass note) are given</li>
    </ul>
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Squeeze function</strong>
</div> 

As a last processing step, we define a function to obtain a dataframe with all the chords of a song squeezed in its various sections (intro, first verse, chorus, second verse...). This will be helpful later in our analysis.

In [ ]:
#The squeeze function returns a dataframe with the all the chords of a song squeezed in a row dependent on a subgroup
# of the section type. Default is "none" and will not filter any type of songs.
#try subgroup="chorus" or subgroup="verse"

def compress(s) :
    return s.dropna().to_list()

def squeeze(df,subgroup="none"):
    df_local=df
    if(subgroup!="none"):
        df_local=df[df["section_type"]==subgroup]
    
    chords = df_local.groupby(["song_id","title"]).chord.agg(compress).reset_index()[['song_id', 'chord']]
    roots = df_local.groupby(["song_id","title"]).root.agg(compress).reset_index()[['song_id', 'root']]
    shorthands = df_local.groupby(["song_id","title"]).shorthand.agg(compress).reset_index()[['song_id', 'shorthand']]
    
    return chords.merge(roots, on = 'song_id').merge(shorthands, on = 'song_id')

In [ ]:
squeeze(collection_df).head()

In [ ]:
#squeeze section allows you to squeeze the sections of songs
def squeeze_section(df,subgroup="none"):
    df_local=df
    if(subgroup!="none"):
        df_local=df[df["section_type"]==subgroup]
    
    chords = df_local.groupby(["song_id","sequence_id","section_type"]).chord.agg(compress).reset_index()
    roots = df_local.groupby(["song_id","sequence_id","section_type"]).root.agg(compress).reset_index()
    shorthands = df_local.groupby(["song_id","sequence_id","section_type"]).shorthand.agg(compress).reset_index()
    
    return chords.merge(roots, on = ['song_id', 'sequence_id', 'section_type']).merge(shorthands, on = ['song_id', 'sequence_id', 'section_type'])

In [ ]:
squeeze_section(collection_df).head()

We can now move to the exploratory analysis.

<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Chord distribution</strong></div>
    <div>
        The first axe of exploration is the most basic one: simply counting. In this section we will look at:
            <ul>
                <li>the most common chords used in the songs</li>
                <li>the distribution of unique chords in the songs</li>
                <li>the roots of the chords</li>
                <li>the quality of the chords</li>
                <li>the bass notes of the chords</li>
                <li>the Bag of Words representation of the songs</li>
            </ul>
    </div>
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Most common chords used</strong>
</div> 

We can start by finding the most used chords in a given section. By default we look at the 50 most common chords.

In [ ]:
def chord_distribution(df, section = 'verse', top = 50):
    title = 'Chord distribution in the ' + section
    plt.figure(figsize = (16,6))
    df[df.section_type == section].chord.value_counts().iloc[:top].plot(kind='bar')
    plt.title(title, fontsize = 20)
    plt.xlabel('Chord', fontsize = 18)
    plt.xticks(rotation = 50)
    plt.ylabel('Number of chords', fontsize = 18)

In [ ]:
chord_distribution(beats_collection_df, 'verse')

In [ ]:
chord_distribution(beats_collection_df, 'chorus')

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Distribution of unique chords</strong>
</div> 

We can also look at the distribution of unique chords used in songs.

In [ ]:
def GetUniqueChords(df):
    return df.groupby('song_id').chord.unique().map(lambda x: len(x))

In [ ]:
def unique_chords_section(df, section = 'verse'):
    unique_chords = GetUniqueChords(df[df.section_type == section])
    n_bins = int(unique_chords.max()) + 1
    unique_chords.plot.hist(bins = n_bins, legend = False)
    title = 'Distribution of the number of unique chords per song in the ' + section
    plt.title(title, fontsize = 20)
    plt.xlabel('Number of unique chords used', fontsize = 18)
    plt.ylabel('Frequency', fontsize = 18)
    plt.show()
    
    print('Statistics:')
    print(unique_chords.describe())
    
    return (unique_chords.describe()['mean'], unique_chords.describe()['std']/np.sqrt(unique_chords.describe()['count']))

In [ ]:
stats_verse = unique_chords_section(beats_collection_df, 'verse')

In [ ]:
stats_chorus = unique_chords_section(beats_collection_df, 'chorus')

In [ ]:
data = [beats_collection_df[beats_collection_df.section_type == 'verse'].groupby('song_id').count().title,
        beats_collection_df[beats_collection_df.section_type == 'chorus'].groupby('song_id').count().title]
plt.figure(figsize = (16,9))
plt.boxplot(data, notch = True, 
                  bootstrap = 5000, 
                  showfliers = True, 
                  showmeans = True)
plt.title('Number of unique chords used in each section', fontsize = 20)
plt.xticks([1,2], ['Verse', 'Chorus'], fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlabel('Section of the song', fontsize = 18)
plt.ylabel('Number of unique chords used', fontsize = 18)
plt.show()

Finally, we'll use statistical tests to find if the differences observed are significant. From the previous plots, we can already expect to not have significant differences here.

In [ ]:
from scipy import stats
import researchpy as rp

We will run an independent t-test to check if the two sample data, i.e. the number of unique chords used in the verse and in the chorus sections of the songs, are the same or not. But first we need to check the independent t-test assumptions:
   * homogeneity of variances (with the Levene's test)
   * normal distributions of redisuals (with Shapiro-Wilk's test)

In [ ]:
stats.levene(data[0], data[1])

In [ ]:
stats.shapiro(data[0] - data[1]) #Need to have the same size, ie only keep songs with both a chorus & a verse

Since both assumptions are met (both p-values are larger than 0.05, meaning the sample data do not violate the assumptions), we can proceed with the independent t-test. Let's start by having a quick look at some basic statistics (where the first line is for verse, the second for chorus and the third for the combined data). As already seen with the plots, means and SDs are nearly the same.

In [ ]:
descriptives, results = rp.ttest(data[0], data[1])
descriptives

In [ ]:
results

As expected, the two sample data do not violate the null hypothesis because p-value=0.8502 > 0.05 (i.e. there are no significant differences).

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> the distribution of the number of unique chords used in each section has ben investigated. For both verse and chorus sections, the distribution is right skewed with a mean slighly under 6 unique chords used and a tail with maximum values reaching up to 30 unique chords used. An independent t-test has been conducted to find if the differences are statistically significant. As expected, results (t[1595]=0.1889, p-value=0.8502 > 0.5) show that the two sample data do not violate the null hypothesis (i.e. the two distributions are similar). 
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Root of the chords</strong>
</div> 

Now we can look at the roots of the chords, first on the whole songs and then on each section.

In [ ]:
root_dist = beats_collection_df.root.value_counts()
#We remove the missing values (marked with an empty space by our parser)
root_dist.drop('', inplace = True)
#Plot
plt.figure(figsize = (16,5))
root_dist.map(lambda x: 100 * x / root_dist.sum()).plot(kind = 'bar')
plt.title('Root distribution of the chords in the whole songs (ordered)', fontsize = 20)
plt.xlabel('Root', fontsize = 18)
plt.xticks(rotation = 'horizontal')
plt.ylabel('Proportion [%]', fontsize = 18)
plt.show()

We can also move to the line of fifths representation to check if we get the characteristic bell shape.

In [ ]:
root_dist = beats_collection_df.root.value_counts()
#Let's move to the line of fifths
fifths_line = ['B#', 'E#', 'A#', 'D#', 'G#', 'C#', 'F#', 
               'B', 'E', 'A', 'D', 'G', 'C', 'F', 
               'Bb', 'Eb', 'Ab', 'Db', 'Gb', 'Cb', 'Fb']
root_dist = pd.Series(index = fifths_line, data = root_dist).fillna(0)
#Plot
plt.figure(figsize = (16,5))
root_dist.map(lambda x: 100 * x / root_dist.sum()).plot(kind = 'bar')
plt.title('Root distribution of the chords in the whole songs (line of fifths)', fontsize = 20)
plt.xlabel('Root', fontsize = 18)
plt.xticks(rotation = 'horizontal')
plt.ylabel('Proportion [%]', fontsize = 18)
plt.show()

In [ ]:
def root_distribution_section(df, section = 'verse', root_repr = 'root', index = 1, fifths = True):
    root_dist = df[df.section_type == section][root_repr].value_counts()
    if fifths:
        root_dist = pd.Series(index = fifths_line, data = root_dist).fillna(0)
    else:
        #We remove the non-roots notation elements
        if '' in root_dist.index:
            root_dist.drop('', inplace = True) 

    #Plot
    plt.figure(figsize = (16,9))
    plt.subplot(2, 1, index)
    root_dist.map(lambda x: 100 * x / root_dist.sum()).plot(kind = 'bar')
    title = 'Root distribution of the chords in the ' + section
    if fifths:
        title += ' (line of fifths)'
    else:
        title += ' (ordered)'
    plt.title(title, fontsize = 20)
    plt.xlabel('Root', fontsize = 18)
    plt.xticks(rotation = 'horizontal')
    plt.ylabel('Proportion [%]', fontsize = 18)
    plt.show()

Let's first plot the results on a normal scale from the most used to the least used roots.

In [ ]:
root_distribution_section(beats_collection_df, 'verse', 'root', 1, False)
root_distribution_section(beats_collection_df, 'chorus', 'root', 2, False)

And then using again the line of fifths.

In [ ]:
root_distribution_section(beats_collection_df, 'verse', 'root', 1)
root_distribution_section(beats_collection_df, 'chorus', 'root', 2)

Finally, we can also show the relative-to-the-tonic roots distribution.

In [ ]:
root_distribution_section(beats_collection_df, 'verse', 'relative_root_tpc', 1, False)
root_distribution_section(beats_collection_df, 'chorus', 'relative_root_tpc', 2, False)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> the distributions of roots (in either versions, relative or not) look fairly similar, with differences in the order of 1%. It is interesting to note that, when using the line of fifths order, we find the characteristic bell-shaped distribution. 
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Quality of the chords</strong>
</div> 

Next we can investigate the quality (or type) of the chords, using the parser previously defined.

In [ ]:
quality_dist = beats_collection_df.shorthand.value_counts()
#We remove the missing values (marked with an empty space by our parser)
quality_dist.drop(['', '*', '&pause'], inplace = True)
#Plot
plt.figure(figsize = (16,5))
quality_dist.map(lambda x: 100 * x / quality_dist.sum()).plot(kind = 'bar')
plt.title('Quality distribution of the chords in the whole songs', fontsize = 20)
plt.xlabel('Quality', fontsize = 18)
plt.xticks(rotation = 45, fontsize = 14)
plt.ylabel('Proportion [%]', fontsize = 18)
plt.show()

In [ ]:
def quality_distribution_section(df, section = 'verse'):
    quality_dist = df[df.section_type == section].shorthand.value_counts()
    #We remove the non-roots notation elements
    extra_notations = ['', '*', '&pause']
    for notation in extra_notations:
        if notation in quality_dist.index:
            quality_dist.drop(notation, inplace = True) 

    #Plot
    plt.figure(figsize = (16,9))
    plt.subplot(2, 1, 1)
    quality_dist.map(lambda x: 100 * x / quality_dist.sum()).plot(kind = 'bar')
    title = 'Quality distribution of the chords in the ' + section
    plt.title(title, fontsize = 20)
    plt.xlabel('Quality', fontsize = 18)
    plt.xticks(rotation = 45, fontsize = 14)
    plt.ylabel('Proportion [%]', fontsize = 18)
    plt.show()

In [ ]:
quality_distribution_section(beats_collection_df, 'verse')
quality_distribution_section(beats_collection_df, 'chorus')

We observe that major chords are the most used by a large margin. Then come in a close range minor, seventh and minor seventh chords. All other chords have much lower frequencies of usage in both verse and chorus sections. Let's compare the mean proportion of these four predominants chord qualities between the two sections.

In [ ]:
def GetCountsOfQuality(df, section, quality):
    return df[df.section_type == section][df.shorthand == quality].groupby('song_id').count().title

In [ ]:
labels = ['Major in verse', 'Major in chorus',
          'Minor in verse', 'Minor in chorus',
          'Seventh in verse', 'Seventh in chorus',
          'Minor Seventh in verse', 'Minor Seventh in chorus']
data = [GetCountsOfQuality(beats_collection_df, 'verse', 'maj'), 
        GetCountsOfQuality(beats_collection_df, 'chorus', 'maj'),
        GetCountsOfQuality(beats_collection_df, 'verse', 'min'),
        GetCountsOfQuality(beats_collection_df, 'chorus', 'min'),
        GetCountsOfQuality(beats_collection_df, 'verse', '7'),
        GetCountsOfQuality(beats_collection_df, 'chorus', '7'),
        GetCountsOfQuality(beats_collection_df, 'verse', 'min7'),
        GetCountsOfQuality(beats_collection_df, 'chorus', 'min7')]
plt.figure(figsize = (16, 9))
plt.boxplot(data,
            notch = True,
            bootstrap = 5000,
            showfliers = False, #Don't show outliers to better compare medians between the two sections
            showmeans = True)
plt.xticks(range(1,9), labels, rotation = 14)
plt.title('Boxplots of frequencies of usage of the four most common chord qualities, per section', fontsize = 20)
plt.xlabel('Chord qualities and section', fontsize = 18)
plt.ylabel('Frequency of usage', fontsize = 18)
plt.show()

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> the most striking result is certainly the overwhelming presence of major chords in the corpus, representing more than half of all the chords (followed by minor chords around 15%). However the differences between verse and chorus chords are fairly small again. We plot the four most common chord qualities (major, minor, seventh and minor seventh), representing more than 80% of our corpus, using boxplots to better compare between the two sections. According to the notches on the boxplots, some differences might indeed be statistically significant but further statistical tests would be needed to verity that.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Bass note of the chords</strong>
</div> 

Finally, the third element of a chord that we can observe is the bass note used (or the lack of it, in most cases).

In [ ]:
bass_note_dist = beats_collection_df.bass.value_counts().rename({'':'No bass note'})
#Plot
plt.figure(figsize = (16,5))
bass_note_dist.map(lambda x: 100 * x / bass_note_dist.sum()).plot(kind = 'bar')
plt.title('Bass note distribution of the chords in the whole songs', fontsize = 20)
plt.xlabel('Bass note', fontsize = 18)
plt.xticks(rotation = 'horizontal')
plt.ylabel('Proportion [%]', fontsize = 18)
plt.show()

As we can see, most chords do not use a bass note. We will therefore exclude this case in order to better observe which notes are more commonly used as bass notes.

In [ ]:
def bassnote_distribution_section(df, section = 'verse'):
    bass_note_dist = df[df.section_type == section].bass.value_counts()
    #We remove the non-roots notation elements
    if '' in bass_note_dist.index:
        bass_note_dist.drop('', inplace = True) 

    #Plot
    plt.figure(figsize = (16,9))
    plt.subplot(2, 1, 1)
    bass_note_dist.map(lambda x: 100 * x / bass_note_dist.sum()).plot(kind = 'bar')
    title = 'Bass note distribution of the chords in the ' + section
    plt.title(title, fontsize = 20)
    plt.xlabel('Bass note', fontsize = 18)
    plt.xticks(rotation = 'horizontal')
    plt.ylabel('Proportion [%]', fontsize = 18)
    plt.show()

In [ ]:
bassnote_distribution_section(beats_collection_df, 'verse')
bassnote_distribution_section(beats_collection_df, 'chorus')

As we did for chord qualities, we will investigate more in depth the frequencies of usage of the most common bass notes, in this case note 5 and note 3.

In [ ]:
def GetCountsOfBassNote(df, section, bass_note):
    return df[df.section_type == section][df.bass == bass_note].groupby('song_id').count().title

In [ ]:
labels = ['Note 5 in verse', 'Note 5 in chorus',
          'Note 3 in verse', 'Note 3 in chorus']
data = [GetCountsOfBassNote(beats_collection_df, 'verse', '5'), 
        GetCountsOfBassNote(beats_collection_df, 'chorus', '5'),
        GetCountsOfBassNote(beats_collection_df, 'verse', '3'),
        GetCountsOfBassNote(beats_collection_df, 'chorus', '3')]
plt.figure(figsize = (16, 9))
plt.boxplot(data,
            notch = True,
            bootstrap = 5000,
            showfliers = True, #Here the outliers do not skew too much the boxplots
            showmeans = True)
plt.xticks(range(1,5), labels, rotation = 14)
plt.title('Boxplots of frequencies of usage of the two most common bass notes, per section', fontsize = 20)
plt.xlabel('Bass note and section', fontsize = 18)
plt.ylabel('Frequency of usage', fontsize = 18)
plt.show()

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> As expected, most chords simply do not have a bass note. It seems this is more of an exception. Here, there seem to be some differences between the verse and the chorus, and again we use boxplots to better investigate the two most common bass notes used (notes 5 and 3). According to the notches, the differences don't seem significant but further tests can here again be used to assessed that.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>BoW representation</strong>
</div> 

Here we try to move to a more advanced analysis, trying to analyze and compare songs and sections of songs as a whole. Let's start by creating a function to compute the Bag of Words representation of each song (or part of a song). In this case, the words will be all the possible chords.

In [ ]:
vocab = dict.fromkeys(beats_collection_df.chord.unique(), 0) 

Let's remove the non-chords elements.

In [ ]:
for non_chord in ['', '&pause', 'nan']:
    vocab.pop(non_chord, None)

In [ ]:
def createBOW(chord_list):
    bow = vocab
    for chord in chord_list:
        if chord in vocab.keys():
            bow[chord] += 1
    #Normalise the BoW to get a distribution
    total = sum(bow.values())
    bow = {chord: count/total for chord, count in bow.items()}
    return np.array(list(bow.values()))

We can then start to compute the BoW representation of the whole songs, using the function squeeze() previously created to help us.

In [ ]:
df_bow = squeeze(collection_df)
df_bow['bow'] = df_bow.chord.map(lambda x: createBOW(x))

In [ ]:
df_bow.head()

Let's check each bow value is indeed a distribution (ie the sum of the elements is 1).

In [ ]:
df_bow.bow.map(lambda x: sum(x)).sort_values()

We will do the same for the chorus and the verse section of songs.

In [ ]:
df_bow_verse = squeeze(collection_df, 'verse')
df_bow_verse['bow'] = df_bow_verse.chord.map(lambda x: createBOW(x))

In [ ]:
df_bow_verse.head()

In [ ]:
df_bow_chorus = squeeze(collection_df, 'chorus')
df_bow_chorus['bow'] = df_bow_chorus.chord.map(lambda x: createBOW(x))

In [ ]:
df_bow_chorus.head()

### Mean distance

We can now compare songs and sections of songs between them. The first idea is to compute the mean distance between all BoW representations of a section and compare it to the mean distance between all BoW representations of whole songs. Our assumption is that, if chorus and verse have specific characteristics, they should be rather similar from songs to songs and therefore have a smaller mean distance compared to the one of whole songs.

In [ ]:
from scipy.spatial import distance

In [ ]:
def ComputeMeanDistance(df, distance):
    dist_sum = 0
    count = 0
    for i in range(len(df)):
        for j in range(i):
                dist_sum += distance(df.bow.iloc[i], df.bow.iloc[j])
                count += 1
    return dist_sum / count                

In [ ]:
ComputeMeanDistance(df_bow, distance.euclidean)

In [ ]:
ComputeMeanDistance(df_bow_verse, distance.euclidean)

In [ ]:
ComputeMeanDistance(df_bow_chorus, distance.euclidean)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Using the Euclidean distance, we have found that indeed the mean distance of verse sections and chorus sections is smaller than the one of whole songs. Of course further tests would be needed to assert that the differences are statistically significant but this is a promising start and could be further explored. Other distance measures and similarities could be tested. In a similar way, we could investigate whether chords used in chorus sections are a subset of chords used in verse sections (using the Jaccard similarity for instance).
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Markov models</strong></div>
    <div>
        The second axe of exploration we have tried is a Markov model approach. In this section we will:
            <ul>
                <li>create the Markov models (for whole songs or only specific sections)</li>
                <li>test the Markov models with differents basic elements (the chords, the roots of the chords, the quality of the chords</li>
                <li>compare them</li>
            </ul>
    </div>
</div>

In [ ]:
from collections import Counter

In [ ]:
###### Circle of fifth

fifth=["F","C","G","D","A","E","B"]
big_fifth=["F","C","G","D","A","E","B"]

for i in range(0,len(fifth)) :
    
    big_fifth.append(fifth[i]+"#")
    big_fifth.insert(i,fifth[i]+"b")
    
######
# Compute bigrams

def bigrams_seq(seq):
    l=list(zip(seq[:-1], seq[1:]))
    return l


def bigrams_corpus(seqs):
    return [bg for seq in seqs for bg in bigrams_seq(seq)]


def markov_heatmap(bigram_seq, order=big_fifth, display_N=False, hide_diag=False) :
    """Take as intput a sequence of bigram in a counter type element and returns a heatmap
    order : display the axis in the given order, default is the circle of fifth
    display_N: allows you to display the silences as well noted ""
    hide_diag: don't display identical bigrams if true, default is False """
    #test_count=Counter(bigram_seq)
    test_count=bigram_seq
    
    if(order=='none'):
        x1s    = [ x1  for ((x1, x2), count) in test_count.items() ]
        x2s    = [ x2  for ((x1, x2), count) in test_count.items() ]
        y=pd.Series(x2s).drop_duplicates().sort_values()
        x=pd.Series(x1s).drop_duplicates().sort_values(ascending=False)
    else:
        if(display_N == False):
            y=order
            x=reversed(order)
            #x=order
            
    #df_local=df_store
    df_local=pd.DataFrame(index=x,columns=y)
    
    #if(display_N==False):
        #df_local=df_local.drop(index='')
        #df_local=df_local.drop(columns='')
    
    for ((x1, x2), count) in test_count.items():
        if(x1 != "" and x2 != ""):
            df_local[x1][x2]=count
            if(x1==x2 and hide_diag):
                df_local[x1][x2]=0
        elif(display_N==True):
            df_local[x1][x2]=count

            

    df_local=df_local.fillna(0)
    plt.figure(figsize=(7,7))
    #print(df_local)
    #return df_local
    sns.heatmap(data=df_local,cmap="coolwarm")

In [ ]:
def substract_counts(c1,c2):
    """returns the substracted counters c1-c2"""
    c=Counter()
    for ((x1,x2),count) in c1.items() :
        c[(x1,x2)]=c1[(x1,x2)]-c2[(x1,x2)]
    return c    
    
    
def add_count(y,count):
    count+=Counter(y)
    return
    
    
def top_transition(c, n = 15, remove_equal = True, position = "head"):   
    """Plots the graph of the n most significant transition
    c: counter element.
    n: Number of elements to display 
    remove_equal: If True removes the transitions of two identical element
    position: Gives the head or tail elements"""
    
    for ((x1,x2),count) in c.items() :
        if(x1==x2 and remove_equal):
               c[(x1,x2)]=0
    
    x1s = [ (x1,x2)  for ((x1, x2), count) in c.items() ]
    values = [ count  for ((x1, x2), count) in c.items() ]
    
    
    df=pd.DataFrame(data=values, index=x1s)
    df=df.reset_index().set_index(0).sort_index(ascending=False).reset_index().set_index('index')
    df=df.rename(columns={0:"%"},index={"index":"bigrams"})
    
    #Plot
    title = 'Most frequent transitions'
    if(position != "tail"):
        df.head(n).plot(kind='bar', legend = False, figsize = (16,6))
    else:
        df.tail(n).plot(kind='bar', legend = False, figsize = (16,6))
        title += ' (tail)'
    plt.title(title, fontsize = 20)
    plt.xticks(rotation = 45, fontsize = 12)
    plt.xlabel('Transitions', fontsize = 18)
    plt.ylabel('Occurences [%]', fontsize = 18)
    plt.show()
    
    return 
    
    
def normalize_counter(c, remove_equal=True):
    
    tot=0
    for ((x1,x2),count) in c.items() :
        if(remove_equal and x1==x2 ):
            tot+=0
        else :
            tot+=count
    
        
    if(tot==0):
        return "error : empty counter"
    
    for ((x1,x2),count) in c.items() :
        c[(x1,x2)]=(c[(x1,x2)]*100)/tot
    return
        
    
    
def count_bigrams(df, chord_representation = 'chord', subsection='none', normalize=False):
    """Counts the bigrams of a given given data frame "df" grouped relative to the subsection.
    if normalize is true the output will be normalized (useful for comparing results)"""
    
    df = squeeze(df, subsection)
    
    df["bigrams"] = df[chord_representation].map(lambda y : bigrams_seq(y))
    c = Counter()
   
    df.bigrams.map(lambda y: add_count(y,c))
    if(normalize):
        normalize_counter(c)
        
    return c 

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Markov model with chords</strong>
</div> 

We can start by creating a Markov model where the basic elements are the chords. The problem is that with nearly 1000 different chords the number of possible transitions is huge (around $1000^{1000}$) and due to the relative small size of our dataset the results will be increadibly sparse (we will only show the first heatmap to illustrate that).  

#### Verse:

In [ ]:
count_verse_chords = count_bigrams(df = beats_collection_df, 
                            chord_representation = 'chord', 
                            subsection = 'verse',
                            normalize = True)
markov_heatmap(count_verse_chords, order = 'none', hide_diag = True)
top_transition(count_verse_chords)

#### Chorus:

In [ ]:
count_chorus_chords = count_bigrams(df = beats_collection_df, 
                            chord_representation = 'chord', 
                            subsection = 'chorus',
                            normalize = True)
#markov_heatmap(count_chorus_chords, order = 'none', hide_diag = True)
top_transition(count_chorus_chords)

#### Difference between chord transition of choruses and verses

In [ ]:
dif = substract_counts(count_chorus_chords, count_verse_chords)
#markov_heatmap(dif, order = 'none', hide_diag = True)
top_transition(dif)
top_transition(dif, position = 'tail')

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> The Markov model for chords in itself doesn't tell us much about differences between verse and chorus, since differences are smaller than 1%.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Markov model with roots of the chords</strong>
</div> 

In order to lower the number of basic elements, we can look at just the roots of the chords used.

#### Verse:

In [ ]:
count_verse_roots = count_bigrams(df = beats_collection_df, 
                            chord_representation = 'root', 
                            subsection = 'verse',
                            normalize = True)
markov_heatmap(count_verse_roots, hide_diag = True)
top_transition(count_verse_roots)

#### Chorus:

In [ ]:
count_chorus_roots = count_bigrams(df = beats_collection_df,
                                   chord_representation = 'root', 
                                   subsection = 'chorus', 
                                   normalize = True)
markov_heatmap(count_chorus_roots, hide_diag = True)
top_transition(count_chorus_roots)


#### Difference between root chord transition of choruses and verses

In [ ]:
dif = substract_counts(count_chorus_roots, count_verse_roots)
markov_heatmap(dif, hide_diag = True)
top_transition(dif)
top_transition(dif, position = 'tail')

#### Bonus : Difference between chord transition of choruses and intros

In [ ]:
count_intro_roots = count_bigrams(df = beats_collection_df,
                                  chord_representation = 'root', 
                                  subsection = 'intro', 
                                  normalize = True)
dif=substract_counts(count_chorus_roots, count_intro_roots)
markov_heatmap(dif,hide_diag=True)
top_transition(dif)
top_transition(dif,position="tail")

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Looking just at the roots is certainly better but the differences are still fairly small. Notice how the diagonal shape is present when using the line of fifths order.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Tonic distance and Musical path</strong></div>
    <div>
        The last axe of exploration revolves around the idea of musical path and tonic distance: how the song evolves through the verse and the chorus. In this section we will look at:
            <ul>
                <li>the chorus/verse proportion in songs</li>
                <li>the tonic proportion</li>
                <li>the per-chord tonic distance</li>
                <li>the musical path in the songs (testing with different distance measures)</li>
            </ul>
    </div>
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Chorus/verse proportion</strong>
</div> 

We consider the distribution of the proportion of beats in a song that belong to choruses, respectively to verses.

In [ ]:
TOTAL_WEIGHT = "total_weight"

d_collection_df[TOTAL_WEIGHT] = d_collection_df.apply(\
    lambda row: weight_row(row[METRE], row[DURATION], row[REPETITION]),axis=1)

d_collection_df[N_SUCC_BEATS] =  d_collection_df.apply(\
    lambda row: successive_repetition_row(row[METRE], row[DURATION]),axis=1)

d_collection_df = d_collection_df.astype({TOTAL_WEIGHT: 'Int64',N_SUCC_BEATS: 'Int64'})

In [ ]:
n_beat_chorus = d_collection_df[d_collection_df.section_type == "chorus"].groupby("song_id")[TOTAL_WEIGHT].sum()\
.reset_index().rename(columns= {TOTAL_WEIGHT:"chorus_weight"})
n_beat_verse = d_collection_df[d_collection_df.section_type == "verse"].groupby("song_id")[TOTAL_WEIGHT].sum()\
.reset_index().rename(columns= {TOTAL_WEIGHT:"verse_weight"})
n_beat = d_collection_df.groupby("song_id")[TOTAL_WEIGHT].sum()

proportions_df = n_beat_chorus.merge(n_beat_verse,on="song_id").merge(n_beat,on="song_id")

proportions_df["chorus_weight"] = proportions_df["chorus_weight"]/proportions_df["total_weight"]
proportions_df["verse_weight"] = proportions_df["verse_weight"]/proportions_df["total_weight"]

In [ ]:
proportions_df[["chorus_weight","verse_weight"]].describe().drop("count")

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> we can observe that the distributions of chorus and verse within songs share similar statistics, both accounting for a third of the songs.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Tonic proportion</strong>
</div> 

In [ ]:
d_collection_df[[SONG_ID,"tonic"]].drop_duplicates().groupby("tonic")\
.count().rename(columns = {SONG_ID:"number_of_songs"}).plot.bar(figsize = (16,6), legend = False)
plt.title('Tonic proportion of the songs in the corpus', fontsize = 20)
plt.xticks(rotation = 'horizontal', fontsize =  14)
plt.xlabel('Tonic', fontsize = 18)
plt.ylabel('Number of songs', fontsize = 18)
plt.show()

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Per-chord tonic distance analysis</strong>
</div> 

In [ ]:
#From now, we consider d_collection_df, that contains only songs that share a chorus and a verse

def per_chord_tonic_distance_analysis(distance,df,n_beat_type,btype):
    """
    Create weighted mean and std of chord distance to tonic and describe them
    
    distance: measure of distance, for exemple relative tpc from the tonic
    df: dataframe to consider
    n_beat_type: dataframe that give for each song the number of beat (= number of chords) for given type
    btype: name of the column in n_beat_type
    """
    
    df["weight*dist"] =df[distance]*df[TOTAL_WEIGHT]
    mean_numerator_df = df.groupby(SONG_ID)["weight*dist"].sum().reset_index()\
    .rename(columns = {"weight*dist":"mean_numerator"})
    
    
    agg_df = mean_numerator_df.merge(n_beat_type, on = "song_id")
    agg_df["mean_distance"] = agg_df["mean_numerator"]/agg_df[btype]
    
    #Need mean to compute variance
    df = df.merge(agg_df[["mean_distance","song_id"]], on = "song_id")
    
    df["variance_item"] = (df["weight*dist"] - df[TOTAL_WEIGHT]*df["mean_distance"])\
    .apply(lambda x : np.power(x,2))
    variance_numerator_df = df.groupby(SONG_ID)["variance_item"].sum().reset_index()\
    .rename(columns = {"variance_item":"variance_numerator"})
    
    agg_df = agg_df.merge(variance_numerator_df, on = "song_id")
    agg_df["variance_distance"] = agg_df["variance_numerator"]/agg_df[btype]
    agg_df["std_distance"] = agg_df["variance_distance"].apply(np.sqrt)
    
    return agg_df.describe()[["mean_distance","std_distance"]]
    
    return mean_distance

In [ ]:
def plot_per_chord_tonic_distance_analysis(distance,df):
    chorus_distance_df = df[df.section_type == "chorus"]
    verse_distance_df = df[df.section_type == "verse"]
    
    n_beat_chorus = df[df.section_type == "chorus"].groupby("song_id")[TOTAL_WEIGHT].sum()\
    .reset_index().rename(columns= {TOTAL_WEIGHT:"chorus_weight"})
    n_beat_verse = df[df.section_type == "verse"].groupby("song_id")[TOTAL_WEIGHT].sum()\
    .reset_index().rename(columns= {TOTAL_WEIGHT:"verse_weight"})
    
    print("Choruses")
    print(per_chord_tonic_distance_analysis(distance,chorus_distance_df,n_beat_chorus,"chorus_weight"))
    print()
    print("Verses")
    print(per_chord_tonic_distance_analysis(distance,verse_distance_df,n_beat_verse,"verse_weight"))

In [ ]:
plot_per_chord_tonic_distance_analysis("relative_root_tpc", d_collection_df)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Results shows that the mean tpc distance to the tonic is higher in chorus sections than in verse sections. Statistical significance of this result should be confirmed, but it is an interesting path. We might perform further exploration by choosing a different measure of distance, for example related to how we perceive chords or to the tonal hierarchy. 
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Musical path analysis</strong>
</div> 

Here we explore the melodic lines of music by investigating how the distance to tonic evolve along the time dimension.

In [ ]:
### Squeeze with repetition
def weighted_squeeze(df,col):
    """
    Recreate a line with one chord per beat
    """
    df["chord_sublist"] = df.apply(lambda row: row[N_SUCC_BEATS]*[row[col]],axis = 1)
    
    line_df = df.groupby([SONG_ID,TYPE,LINE_NUMBER,])["chord_sublist"].sum().reset_index()
    
    
    #line_df["chord_sublist"] = line_df["chord_sublist"].apply(lambda l: np.array(l).flatten())
    
    
    return line_df.rename(columns = {"chord_sublist":"chord_list"})
    
def full_squeeze(df,col):
    """
    Recreate a section, considering repetitions of full lines.
    """
    #Not implemented yet
    return 0

In [ ]:
line_df = weighted_squeeze(d_collection_df,"relative_root_tpc")

In [ ]:
def mean_path(line_df,line_length,verbose=True):
    """
    Show mean path for given line length
    """
    
    f = plt.figure(figsize=(12,4))
    
    means_dic = {}
    
    for i, section_type in enumerate(("chorus","verse")):
        
        f.add_subplot(1,2,i+1)
        
        sample_df = line_df[line_df["chord_list"].apply(lambda l: len(l) == line_length)]

        sample_df = sample_df[sample_df[TYPE] == section_type]
        
        means = pd.DataFrame(sample_df["chord_list"].values.tolist()).mean(axis = 0)
        means_dic[section_type] = means
        
        if verbose:
            print(len(sample_df),"lines considered for",section_type)

            means.plot.bar(figsize = (16,4))
            plt.title("Average relative root tpc for lines of length {} in {}".format(line_length,section_type), fontsize = 16)
            plt.xlabel("Position of chord", fontsize = 14)
            plt.xticks(rotation = 'horizontal', fontsize = 12)
            plt.ylabel("Avg relative root tpc", fontsize = 14)
    
    
    if not verbose:
        return means_dic    

In [ ]:
mean_path(line_df,8)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Those graphs concentrate only on melodic lines of length 8, because they were numerous. However a normalization that would allow us to observe aggregations of melodic lines of all length still need to be found. This should therefore be considered more as an illustration than a fully representative graph.

We observe the same result as the previous one: the path have a similar shape, but the pitch class is higher in the choruses. 
</div>

In [ ]:
def plot_line_path(df,distance,line_len):
    line_df = weighted_squeeze(df,distance)
    mean_path(line_df,line_len)
    
def plot_section_path(df,distance,sec_len):
    sec_df = full_squeeze(df,distance)
    mean_path(sec_df,sec_len)

### Tests with different distances

#### Distance between tonic and root of chord, in number of semitones up or down

In [ ]:
d_collection_df["tpc_distance"] = d_collection_df["relative_root_tpc"].apply(lambda tpc: min(tpc,12-tpc))

In [ ]:
plot_per_chord_tonic_distance_analysis("tpc_distance",d_collection_df)

In [ ]:
plot_line_path(d_collection_df,"tpc_distance",8)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Here, we compute a real distance in semitones between the tonic and the root of the chords, in the context of octave equivalence. This leads to the same conclusion: the distance to tonic looks larger in the choruses.
</div>

#### Concordance distance (manual mapping)

This small proximity attribution is done following the proximity graph of course 9 (slide 27)

In [ ]:
concordance_dic = {0:15,7:5,5:2.5,9:2.5,1:2.5}

In [ ]:
d_collection_df["concordance_proximity"] = d_collection_df["relative_root_tpc"]\
.apply(lambda r: concordance_dic.get(r))

d_collection_df = d_collection_df.fillna({"concordance_proximity":1}).dropna(subset = ["relative_root_tpc"])

In [ ]:
plot_per_chord_tonic_distance_analysis("concordance_proximity",d_collection_df)

In [ ]:
plot_line_path(d_collection_df,"concordance_proximity",8)

<div style="border:1px solid black; padding:10px 10px; background-color: #FFAEBC;">
    <strong>Conclusion:</strong> Again, the proximity is higher in the verses, suggesting a smaller distance to tonic. The shapes of both graphs show again interesting resemblance.
</div>

<div style="border:1px solid black; padding:10px 10px; background-color: #FBE7C6;"> 
    <div style = "font-size:25px; text-align:center;"><strong>Summary of Milestone 3</strong></div>
</div>

For this milestone, we have started by finishing all processing of the dataset in order to obtain a simple dataframe for all further analyses. Each row of the dataframe represents a chord played on a beat, with informations about all components of said chord (its root, its quality and its bass note, if one has been used). These informations have been extracted from the chord notation used in the dataset, defined by http://ismir2005.ismir.net/proceedings/1080.pdf (with some additions specific to this dataset). We have also only kept songs with both a verse and a chorus, resulting in a corpus of 712 songs. 

Once these final processing steps done, we have started our exploratory analysis. We have three main axis: 
* chord distributions
* Markov models
* tonic distance and musical path

<strong>Chord distributions:</strong> For the first axe, most analyses have not been really interesting since differences between verse and chorus are fairly small. We expected to have more differences here but it could be that distributions are too simple of a way to compare the two sections. When moving to more advanced analysis however, using the bag of words representation to compute the mean distance between all chorus sections, all verse sections and whole songs, we found some interesting results. Indeed, it seems that the mean distance computed is smaller for verse and for chorus compared to the one of whole songs. This can be interpretated as chorus and verse having some characteristics across all songs that makes them similar and therefore recognizable. This of course is only a start, statistical tests would be required to confirm that and other distance measures should be used as well for comparison. In particular, the Jaccard similarity could be used to investigate whether chorus chords are a subset of verse chords.

<strong>Markov models:</strong> When moving to the Markov models, results are quite difficult to interpret since differences are very small (less than 1%) and transitions from one chord to another are rather abstract. It is not that clear what further steps could be taken in this direction to get better results.

<strong>Tonic distance and Musical path:</strong> Finally, the last axe has perhaps proven to be the most interesting. The idea is to define a tonic distance (such as the number of semitones between the chord root and the tonic of the song) and measure it over the song. Results show that the mean tonic distance seems higher in the chorus than in the verse. This relates to this idea of musical home, where composers adventure themselves further away from it in the chorus. This has led us to the concept of musical path, where we have investigated how the distance to tonic evolves along the time dimension. The analyses have been conducted on musical lines of length 8 and, surprisingly, when plotting the average  relative root tpc as a function of the position of the chord in the musical line, very similar shapes are found for verse and chorus sections. This is confirmed with different distance measures. This is certainly interesting enough to be pursued, especially by normalizing the results and testing the statistical significance of the differences observed.

One thing we have not explored is the evolution over the years of songs. Since most of the results we have found are difficult to interpret or rely on very small differences, applying a time dimension over it and therefore reducing even further the size of the data sample for each analysis doens't seem to be the right way to go. However we plan to at least test this direction for our more interesting results, as it could still reveal itself interesting.

<div style="border:1px solid black; padding:10px 10px; background-color: #A0E7E5; text-align: center; font-size:20px;"> 
    <strong>Final analysis for musical path analysis</strong>
</div> 

In [ ]:
beats_collection_df = beats_collection_df.astype({"relative_root_tpc":"Int64"})

In [ ]:
def temporal_squeeze(beats_collection_df, column, type, measurement = "relative_root_tpc"):
    
    selected_df = beats_collection_df[beats_collection_df.section_type == type]
    
    agg_df = selected_df.groupby(["song_id",column])[measurement].apply(list).reset_index()
    
    return agg_df

In [ ]:
chorus_line_df = temporal_squeeze(beats_collection_df,"line_id","chorus")
chorus_section_df = temporal_squeeze(beats_collection_df,"sequence_id","chorus")
verse_line_df = temporal_squeeze(beats_collection_df,"line_id","verse")
verse_section_df = temporal_squeeze(beats_collection_df,"sequence_id","verse")

## Heatmap of musical path

### Most common line and section length

The histograms were really sparse, so we display just the most common lengths

In [ ]:
def show_most_common(df, column, type, measurement="relative_root_tpc"):
    
    df["length"] = df[measurement].apply(len)
    
    print("*Number of {} per length for {}*".format(column,type))
    print(df.groupby("length")[column].count().sort_values(ascending = False).head(10)\
     .reset_index().rename(columns = {column:"Quantity"}))
    print()

In [ ]:
show_most_common(chorus_line_df,"line_id","chorus")
show_most_common(verse_line_df,"line_id","verse")
show_most_common(chorus_section_df,"sequence_id","chorus")
show_most_common(verse_section_df,"sequence_id","verse")

Observation: There are only a few lengths that occur enough to give meaningful results. We will in a first time consider the lengths for which there are more than 100 rows in both chorus and verse type. This means the 5 first lengths for the lines and the 3 first for choruses.

Note also that we perform global comparison. We are not interested in comparing within one song the difference between the verses and the choruses, as most of the songs probably have verses and choruses of different lengths.

### Heatmaps

In [ ]:
line_lengths = (16,8,12,32,20)
section_lengths = (32,64,48)

In [ ]:
def create_heatmap_df(df,length,measurement = "relative_root_tpc"):
    
    selected_df = df[df.length == length]
    
    heatmap_df = pd.DataFrame()
    
    for i in range(length):
        
        pname = "pos {}".format(i)
        
        selected_df[pname] = selected_df[measurement].apply(lambda l: l[i])
        selected_df = selected_df.astype({pname:"Int64"})
        
        heatmap_df[pname] = selected_df[pname].value_counts()
        
        #Normalisation
        heatmap_df = heatmap_df/heatmap_df.sum(axis = 0)
        
        heatmap_df.index.name = measurement
        
    return heatmap_df.sort_index(ascending=False)
    
        
def create_heatmaps(chorus_df,verse_df,length,measurement = "relative_root_tpc"):
    """
    Plot heatmaps for choruses, verses and difference
    """
    heatmap_chorus = create_heatmap_df(chorus_df,length)
    heatmap_verse = create_heatmap_df(verse_df,length)
    heatmap_diff = (heatmap_chorus - heatmap_verse).sort_index(ascending=False)
    
    sns.heatmap(heatmap_chorus)
    plt.title("Pitch class per position for choruses of length {}".format(length))
    plt.show()
    
    sns.heatmap(heatmap_verse)
    plt.title("Pitch class per position for verses of length {}".format(length))
    plt.show()
    
    sns.heatmap(heatmap_diff)
    plt.title("Difference in pitch class occurences (length {})".format(length))
    plt.show()
    
    return 0
    

In [ ]:
for i in line_lengths:
    create_heatmaps(chorus_line_df,verse_line_df,i)

In [ ]:
for i in section_lengths:
    create_heatmaps(chorus_section_df,verse_section_df,i)

## Overlaying graphs

In [ ]:
# TODO

## Tonic distance in individual songs

In [ ]:
# TODO

In [ ]:
test = create_heatmap_df(chorus_line_df,16)

In [ ]:
create_heatmap_df(chorus_line_df,32)